In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from kiwipiepy import Kiwi
from tqdm import tqdm
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
# df = pd.read_csv("/data/ephemeral/home/yujin/processed_train_4220.csv")
df = pd.read_csv("/data/ephemeral/home/level2-nlp-datacentric-nlp-10/train_denoised_none.csv")

df = df.dropna(subset=['text'])

In [ ]:
kiwi = Kiwi()

In [ ]:
# 텍스트를 형태소 단위로 나누고 필요한 품사만 필터링
def preprocess_text(text):
    tokens = kiwi.tokenize(text)
    filtered_tokens = [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA', 'XR']]

    # # 영어 토큰화
    # english_tokens = word_tokenize(text)  # NLTK를 사용하여 영어 단어 토큰화
    # # 영어 단어는 필요에 따라 추가적인 필터링을 할 수 있습니다. 예: 영문 단어 길이가 1 이상인 경우
    # english_tokens = [word for word in english_tokens if len(word) > 1]
    
    # # 한국어와 영어 토큰 결합
    # combined_tokens = filtered_tokens + english_tokens
    # return ' '.join(combined_tokens)
    return ' '.join(filtered_tokens)

# 전체 데이터에 대해 전처리 적용
df['processed_text'] = df['text'].apply(preprocess_text)
df.head()

In [ ]:
# 한국어 stop words 리스트 설정 (예시)
korean_stopwords = [
    '을', '를', '는', '에', '의', '다', '있다', '하다', 
    '그', '저', '그들', '저희', '이런', '어떤', '모든', '자신', '지금', 
    '또', '이렇게', '그런', '그리고', '하지만', '그러나', '또한', '때문에'
]

# CountVectorizer로 텍스트를 벡터화
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=korean_stopwords)
dtm = vectorizer.fit_transform(df['processed_text'])

In [ ]:
# LDA 모델 학습
num_topics = 7  # 주제의 개수를 설정 (예: 카테고리가 7개인 경우)
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(dtm)

In [ ]:
# 각 문서에 대해 주제 확률 추출
df['topic'] = lda_model.transform(dtm).argmax(axis=1)

# 결과 확인
df

In [ ]:
# 주제별 상위 단어 확인
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}: ", " ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(lda_model, vectorizer.get_feature_names_out(), 10)

In [ ]:
# pyLDAvis 시각화
pyLDAvis.enable_notebook()

# 직접 LDA 결과를 시각화
panel = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_ / lda_model.components_.sum(axis=1)[:, None],
    doc_topic_dists=lda_model.transform(dtm),
    doc_lengths=[len(doc.split()) for doc in df['processed_text']],
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=dtm.sum(axis=0).getA1()
)
pyLDAvis.display(panel)